In [1]:
from ultralytics import YOLO

# Load the YOLOv8n model
model = YOLO('yolov8n.pt')

# Access the class names
class_names = model.names

# Print the class names
print(class_names)


{0: 'Capillary', 1: 'Bubble'}


In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import os

In [ ]:
# Load the YOLO model from a local file
dd = 'C:/Users/f00541990/Documents/litebc/'
# dd = 'C:/Users/f00541990/Downloads'

model = YOLO(os.path.join(dd, 'yolov8n.pt'))

# Access class names (optional)
class_names = model.names
print(class_names)

In [ ]:
import torch
import torchvision.ops.boxes as bops
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random

# Use torchvision for IoU calculation
def iou_torchvision(box1, box2):
    box1 = [box1[2], box1[3], box1[2] + box1[4], box1[3] + box1[5]]
    box2 = [box2[2], box2[3], box2[2] + box2[4], box2[3] + box2[5]]
    
    box1_tensor = torch.tensor([box1], dtype=torch.float32)
    box2_tensor = torch.tensor([box2], dtype=torch.float32)
    
    iou_value = bops.box_iou(box1_tensor, box2_tensor)
    return iou_value.item()

# Function to assign unique IDs based on IoU matching across two lists of bboxes
def assign_bbox_ids(L1, L2, iou_threshold=0.9):
    next_id = 1
    L1_ids = []
    L2_ids = [-1] * len(L2)  # Pre-fill L2 with placeholders
    matched_L2 = set()

    # Assign IDs to L1 and L2
    for i, box1 in enumerate(L1):
        matched = False
        for j, box2 in enumerate(L2):
            if j in matched_L2:
                continue
            if iou_torchvision(box1, box2) >= iou_threshold:
                L1_ids.append(next_id)
                L2_ids[j] = next_id  # Match L2[j] with L1[i]
                matched_L2.add(j)
                matched = True
                next_id += 1
                break
        if not matched:
            L1_ids.append(next_id)
            next_id += 1

    # Assign IDs to unmatched boxes in L2
    for j in range(len(L2)):
        if L2_ids[j] == -1:  # If still unmatched
            L2_ids[j] = next_id
            next_id += 1

    return L1_ids, L2_ids  # Return two separate lists

# Visualization function for bounding boxes in separate subplots
def visualize_bboxes_separate(L1, L2, L1_ids, L2_ids):
    img1 = np.ones((1000, 1000, 3), dtype=np.uint8) * 255
    img2 = np.ones((1000, 1000, 3), dtype=np.uint8) * 255
    colors = {}

    for bbox_id in set(L1_ids + L2_ids):
        colors[bbox_id] = [random.randint(0, 255) for _ in range(3)]

    def draw_bbox(img, bbox, bbox_id, label):
        x, y, w, h = bbox[2], bbox[3], bbox[4], bbox[5]
        color = colors[bbox_id]
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        cv2.putText(img, f"ID: {bbox_id}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        cv2.putText(img, label, (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

    for i, bbox in enumerate(L1):
        draw_bbox(img1, bbox, L1_ids[i], "L1")

    for i, bbox in enumerate(L2):
        draw_bbox(img2, bbox, L2_ids[i], "L2")

    fig, ax = plt.subplots(1, 2, figsize=(10, 10))
    ax[0].imshow(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    ax[0].set_title("L1 Bounding Boxes")
    ax[0].axis('off')

    ax[1].imshow(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    ax[1].set_title("L2 Bounding Boxes")
    ax[1].axis('off')

    plt.show()

# Example bounding boxes for a more challenging scenario
L1 = [
    (0, 1.0, 50, 50, 100, 100),
    (0, 1.0, 550, 550, 80, 80),
    (0, 1.0, 200, 200, 100, 100),
    (0, 1.0, 400, 400, 100, 100),
    
]

L2 = [
    (0, 1.0, 48, 52, 100, 100),  # Slight shift, should match L1[0]
    (0, 1.0, 548, 548, 80, 80),  # Should match L1[3]
    (0, 1.0, 210, 210, 90, 90),  # Slightly different, should match L1[1]
    (0, 1.0, 350, 400, 100, 100) # Should not match any box in L1
]

# Assign IDs and visualize
L1_ids, L2_ids = assign_bbox_ids(L1, L2, iou_threshold=0.8)
visualize_bboxes_separate(L1, L2, L1_ids, L2_ids)


In [ ]:
import rbo

In [ ]:
import rbo

In [ ]:
L1_ids, L2_ids

In [ ]:
rbo.RankingSimilarity(L1_ids, L2_ids).rbo()

In [ ]:
rbo.RankingSimilarity(L1_ids, L2_ids).rbo()

In [ ]:
L1 = [
    (0, 1.0, 100, 100, 200, 200),  # Box in the center
    (0, 1.0, 400, 400, 150, 150)   # Another box on the right
]

L2 = [
    (0, 1.0, 100, 100, 200, 200),  # Exactly the same box
    (0, 1.0, 400, 400, 150, 150)   # Same as L1[1]
]

L1_ids, L2_ids = assign_bbox_ids(L1, L2, iou_threshold=0.8)
res = rbo.RankingSimilarity(L1_ids, L2_ids).rbo()

print(L1_ids, L2_ids)
print(res)

In [ ]:
L1 = [
    (0, 1.0, 50, 50, 100, 100),    # Box near the top-left
    (0, 1.0, 300, 300, 150, 150),  # Another box in the middle
]

L2 = [
    (0, 1.0, 55, 55, 100, 100),    # Slightly shifted box from L1[0]
    (0, 1.0, 305, 310, 140, 150)   # Slightly shifted and resized box from L1[1]
]

L1_ids, L2_ids = assign_bbox_ids(L1, L2, iou_threshold=0.8)
res = rbo.RankingSimilarity(L1_ids, L2_ids).rbo()

print(L1_ids, L2_ids)
print(res)

In [ ]:
L1 = [
    (0, 1.0, 100, 100, 150, 150),   # Box in the middle
    (0, 1.0, 500, 500, 100, 100),   # Smaller box in the bottom-right
]

L2 = [
    
    (0, 1.0, 600, 600, 100, 100),   # No overlap with L1, new box
    (0, 1.0, 90, 90, 50, 50),        # Smaller box slightly outside L1[0]
    (0, 1.0, 110, 110, 140, 140),   # Partial overlap with L1[0]
]

L1_ids, L2_ids = assign_bbox_ids(L1, L2, iou_threshold=0.5)
res = rbo.RankingSimilarity(L1_ids, L2_ids).rbo()

print(L1_ids, L2_ids)
print(res)

In [ ]:
L1 = [
    (0, 1.0, 50, 50, 100, 100),    # Top-left corner box
    (0, 1.0, 400, 200, 150, 100)   # Box in the middle-right
]

L2 = [
    (0, 1.0, 300, 300, 150, 150),  # Different box, no overlap with L1
    (0, 1.0, 600, 400, 200, 150)   # Completely new box
]

L1_ids, L2_ids = assign_bbox_ids(L1, L2, iou_threshold=0.5)
res = rbo.RankingSimilarity(L1_ids, L2_ids).rbo()

print(L1_ids, L2_ids)
print(res)


In [ ]:
x = rbo.RankingSimilarity(L1_ids, L2_ids)
x.p = 1
x.rbo()

In [ ]:
x.p

In [ ]:
dir(x)